Выберите наборы весов и смещений, которые позволят изображенному многослойному перцептрону (в данном случае это сеть из "перцептронов" в понимании предыдущей недели) решать проблему XOR. XOR — логическая функция от двух переменных (которые будут подаваться на входы перцептрону) со следующей таблицей истинности: 

 
У сети, изображённой ниже, все нейроны внутреннего и выходного слоя — это обычные перцептроны, у которых пронумерованы веса (цифры на стрелочках) и смещения (цифры в кружках, соответствующих этим перцептронам). На входном слое — просто входы, всё как обычно. 



Ваша задача — написать в обозначенном нумерацией порядке через запятую числа (с десятичным разделителем-точкой), которые, на ваш взгляд, нужно поместить в соответствующий вес или смещение, чтобы получившаяся сеть, принимая на вход нули и единицы, отдавала на выходе значение XOR для поданных входов.

Пример. Вы считаете, что сеть должна иметь вот такой вид:

Тогда вы пишете в ответ: -4.0, 4.0, 5.0, -1.0, 0.0, 1.0, 4.0, 3.0, 5.0. Мы посчитаем, что выдаёт сетка на ваших весах, и узнаем, правы ли вы.
Подсказка. Мы знаем, что перцептроны-одиночки умеют имитировать поведение простых логических функций вроде AND, OR, NOT. Структура этой сети — два перцептрона от входных данных и ещё один перцептрон от значений первых двух перцептронов. 

In [2]:
import numpy as np #Analysis of the algorithm for reverse distribution

In [4]:
class Perceptron:
    def __init__(self, w, b):
        self.w = w
        self.b = b
        
    def forward_pass(self, single_input):
        result = 0
        for i in range(0, len(self.w)):
            result += self.w[i] * single_input[i]
        result += self.b

        if result > 0:
            return 1
        else:
            return 0

    def train_on_single_example(self, example, y):
#         out = int(self.w.T.dot(example) + self.b > 0) # в матричном виде, без self.forward_pass
        out = self.forward_pass(example)
        error = y - out
        self.w = self.w + error * example
        self.b = self.b + error
        return error
    
    def train_until_convergence(self, input_matrix, y, max_steps=1e8):
        i = 0
        errors = 1
        while errors and i < max_steps:
            i += 1
            errors = 0
            for example, answer in zip(input_matrix, y):
                example = example.reshape((example.size, 1))
                error = self.train_on_single_example(example, answer)
                errors += 0 if error == 0 else 1  # ЗДЕСЬ БЫЛА ОШИБКА

In [5]:
w = np.array([[0,0]]).T
p1 = Perceptron(w.copy(), 0) # a1 = AND(not(a),b)
p2 = Perceptron(w.copy(), 0) # a2 = AND(a, not(b))
p3 = Perceptron(w.copy(), 0) # OR(a1, a2)
X = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])
y1 = np.array([[0, 0, 1, 0]]).T
y2 = np.array([[0, 1, 0, 0]]).T
y3 = np.array([[0, 1, 1, 1]]).T
p1.train_until_convergence(X, y1)
p2.train_until_convergence(X, y2)
p3.train_until_convergence(X, y3)
print(p1.w, p1.b)
print(p2.w, p2.b)
print(p3.w, p3.b)

for example in X:
    print(example)
    a = p1.forward_pass(example)
    b = p2.forward_pass(example)
    print(p3.forward_pass([a, b]))

[[-2]
 [ 1]] [0]
[[ 1]
 [-1]] [0]
[[1]
 [1]] [0]
[0 0]
0
[1 0]
1
[0 1]
1
[1 1]
0


как было предложено ниже:
xor=or(and(!a,b),and(a,!b))
для  and(!a,b):
a | b | !a | and(!a,b)
-------------------------
0 | 0 | 1 | 0
0 | 1 | 1 | 1
1 | 0 | 0 | 0
1 | 1 | 0 | 0
Напишем систему уравнений (x1*w1+x2*w2+b ...):
0*w1+0*w2+b<=0
0*w1+1*w2+b>0
1*w1+0*w2+b<=0
1*w1+1*w2+b<=0

получаем:
b<=0
w2+b>0
w1+b<=0
w1+w2+b<=0

Решаем подбором, результат записываем в ответ: w1,w2,b
Подобны образом составляем таблицу и систему уравнений для and(a,!b), результат записываем (продолжаем) в ответ: ...,w1,w2,b
И, наконец, составляем таблицу и систему уравнений для or(...), результат записываем (продолжаем) в ответ: ...,w1,w2,b
